# 迭代类型

## 迭代器协议

python中的容器都具有迭代功能，可以依次迭代访问容器中的各元素，如下所示：

In [34]:
l = [1, 2, 3, 4]
for i in l:
    print(i)

1
2
3
4


python中，迭代功能实际上是通过迭代器协议（Iterator Protocol）实现的。迭代器协议主要提供了两个内建方法，实现了这两个方法的类即满足迭代器协议。

- `__iter__()`返回一个迭代器对象，该方法返回的对象都实现了`__next__()`方法。通过`iter()`函数，即可以调用迭代器对象的`__iter__()`方法。

- `__next__()`返回容器中的下一项，若已到达最后一项，则抛出`StopIteration`异常。通过`next()`函数，即可调用迭代器对象的`__next__()`方法。

类似的，通过`len()`函数，即可调用对象的`__len__()`方法。

python中，容器对象都需要实现`__iter__()`方法来提供迭代功能。

`container.__iter__()`返回一个迭代器对象，而该对象需要满足迭代器协议，即返回的对象需要实现`__iter__()`与`__next__()`方法。

## `collections.abc.Iterable`与`collections.abc.Iterator`

python中`collections`模块提供了两个与迭代类型相关的抽象基类，分别为可迭代对象与迭代器对象提供了接口。

- `collections.abc.Iterable`为可迭代对象提供接口，定义了`__iter__()`方法。
- `collections.abc.Iterator`为迭代器提供接口，定义了`__iter__()`与`__next__()`方法。

可以通过`isinstance()`判断一个对象是否为可迭代对象或迭代器对象，即实现了`__iter__()`方法的对象为可迭代对象，而实现了`__iter__()`与`__next__()`方法的对象为迭代器对象。

In [2]:
import collections

In [30]:
class NonIterable:
    pass

non_iterable = NonIterable()
isinstance(non_iterable, collections.abc.Iterable)

False

In [31]:
class IsIterable:
    def __iter__(self):
        pass
    
is_iterable = IsIterable()
isinstance(is_iterable, collections.abc.Iterable)

True

In [32]:
class IsIterator:
    def __next__(self):
        pass
    
    def __iter__(self):
        pass

is_iterator = IsIterator()
isinstance(is_iterator, collections.abc.Iterator)

True

`isinstance(obj, Iterable)`可以判断对象是否包含`__iter__()`方法，但对于通过`__getitem__()`方法实现迭代的对象无法判断。因此，判断某个对象是否是可迭代对象的唯一可靠方法是通过`iter(obj)`。

根据`collections.abc.Iterable`与`collections.abc.Iterator`两个抽象基类，可以看出可迭代对象与迭代器对象的区别在于，可迭代对象要求实现`__iter__()`方法，而迭代器对象要求实现`__iter__()`与`__next__()`方法。

||`__iter__()`|`__next__()`|
|-|-|-|
|可迭代对象|返回一个迭代器对象|无|
|迭代器对象|返回迭代器本身，即`self`|返回迭代中的下一项|

## 可迭代对象

python中，`tuple`、`list`、`set`、`dict`、`file`、`string`等对象都是可迭代对象。

一般而言，可迭代对象需要实现了`__iter__()`方法。通过`iter(obj)`方法可以返回一个新的迭代器对象，而`iter(obj)`函数直接调用`obj.__iter__()`方法。

简而言之，凡是可以通过`iter()`函数返回一个迭代器的对象都是可迭代对象。

通过`isinstance(obj, Iterable)`可判断对象`obj`是否为可迭代对象。

In [20]:
# list对象为可迭代对象
isinstance([], collections.Iterable)

True

In [22]:
# dict对象为可迭代对象
isinstance({'a': 1, 'b': 2}, collections.Iterable)

True

In [3]:
# dict对象为可迭代对象
isinstance({'a': 1, 'b': 2}, collections.Iterable)

True

## 迭代器

迭代器对象需要同时实现`__iter__()`与`__next__()`两个方法，

在迭代器对象中，`__iter__()`方法返回是迭代器对象自身。这样的设计思路，保证了可迭代对象与迭代器对象都可以用于在如`for...in`等语句进行迭代访问。

python为不同的可迭代容器对象提供了不同类型的迭代器对象，而这些不同类型的迭代器对象都满足迭代器协议。

通过`type()`函数可以返回对象的类型，可以发现不同的可迭代对象的迭代器类型是不同的。

In [23]:
l = [1, 2, 3]
type(iter(l))

list_iterator

In [24]:
s = {1, 2, 3}
type(iter(s))

set_iterator

In [25]:
d = {'a': 1, 'b': 2}
type(iter(d))

dict_keyiterator

In [26]:
ss = 'abcdefg'
type(iter(ss))

str_iterator

## 自定义可迭代对象

通过迭代器协议，我们可以自定义满足迭代功能的容器对象。

In [4]:
class CustomedIterator:
    def __init__(self, container):
        self.container = container
        self.current_index = 0
    
    def __next__(self):
        if self.current_index < len(self.container):
            self.current_index += 1
            return self.container[self.current_index - 1]
        else:
            raise StopIteration
    
    def __iter__(self):
        return self
        

class CustomedList:
    def __init__(self):
        self.container = []
        
    def __iter__(self):
        return CustomedIterator(self.container)
    
    def add(self, value):
        self.container.append(value)
    
my_container = CustomedList()

for x in range(5):
    my_container.add(x)

for x in my_container:
    print(x)

0
1
2
3
4


定义了一个可迭代类型的容器`CustomedList`，实现了`__iter__()`方法，同时定义了一个迭代器对象`CustomedIterator`，实现了`__iter__()`与`__next__()`方法。

但是这种方式在实际应用中过于臃肿，因此并不常用。通常自定义的容器对象与其迭代器对象可以合二为一，同时定义`__iter__()`与`__next__()`方法，即对象既是可迭代容器对象，也是迭代器对象。其中`__iter__()`方法返回的是`self`，即对象自身。

In [5]:
class NewCustomedList:
    def __init__(self):
        self.container = []
        self.current_index = 0
        
    def __next__(self):
        if self.current_index < len(self.container):
            self.current_index += 1
            return self.container[self.current_index - 1]
        else:
            raise StopIteration
            
    def __iter__(self):
        return self
    
    def add(self, value):
        self.container.append(value)
        
my_container = NewCustomedList()

for x in range(5):
    my_container.add(x)

for x in my_container:
    print(x)

0
1
2
3
4


在以上的示例中，我们还可以通过`iter()`与`next()`函数，直接利用`list`类型的可迭代性实现容器。

In [6]:
class SimpleCustomedList:
    def __init__(self):
        self.container = []
        
    def __next__(self):
        return next(self.container)
    
    def __iter__(self):
        return iter(self.container)
    
    def add(self, value):
        self.container.append(value)
        
my_container = SimpleCustomedList()

for x in range(5):
    my_container.add(x)

for x in my_container:
    print(x)

0
1
2
3
4
